# Homework 3

We first import the needed libraries and set the elements we need for the rest of the Homework.

In [ ]:
# Used to look for the saved data sheets
DATA_FOLDER = 'Data/' #still need to create this

In [ ]:
# Importing libraries
import os
import pandas as pd
import json
import seaborn as sns #might not be useful
import folium

In [ ]:
folium.__version__ == '0.5.0' # just checking if we have all the right one installed

## Unemployment in Europe

The very first thing we had to think about is what data we wanted to use. Eurostat had a lot of types of information, even more so, had information per country, but also per province/state of some countries. Thus, we had to determine what kind of information - and how much - we wanted to show.

Thus, we decided to take the **yearly** average for the unemployment rates and not the monthly ones, thus getting data from 2016 and not from 2017. For the monthly rates, we could have had for the beginning of this year, but we choose against it.

Secondly, seeing as the idea was to compare the unemployment rate of Europe to that of Switzerland, it would be more appropriate to take the data by country rather than that per province, even more so seeing as Switzerland is cut in three in that dataset. So, to be able to compare it to Switzerland as a whole, we should have Switzerland as only one entity.

## Unemployment in Switzerland

### Differences in unemployment between foreigners and natives

### The Röstigraben